In [ ]:
from library_dicom.dicom_processor.tools.pyradiomic import *
import SimpleITK as sitk 
import os 
import numpy as np 
import csv
import json 
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#AHL NIFTI 
csv_path= '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/AHL/AHL2011_dataset_dmax.csv'
with open(csv_path, 'r') as csv_file :
    reader = csv.reader(csv_file, delimiter = ',') #liste pour chaque ligne 
    csv_file_ = []
    for row in reader :
        csv_file_.append(row)
        
del csv_file_[0] #enlever première ligne

In [ ]:
print("Nombre de mask à traiter : ", len(csv_file_))

#### - Keep only pet 0 

In [ ]:
pet0 = []
for row in csv_file_ : 
    if 'pet0' in row : 
        pet0.append(row)
print(len(pet0))

#### - Rewrite abs path 

In [ ]:
base_path = '/media/deeplearning/LACIE SHARE'
for row in pet0 : 
    ct = row[-3]
    pt = row[-2]
    mask = row[-1]
    up_ct = os.path.join(base_path, ct)
    up_pt = os.path.join(base_path, pt)
    up_mask = os.path.join(base_path, mask)
    row[-3] = up_ct 
    row[-2] = up_pt 
    row[-1] = up_mask

In [ ]:
#get date 
csv_path_2= '/media/deeplearning/LACIE SHARE/AHL_NIFTI/pet0.csv'
with open(csv_path_2, 'r') as csv_file :
    reader = csv.reader(csv_file, delimiter = ';') #liste pour chaque ligne 
    csv_file_2 = []
    for row in reader :
        csv_file_2.append(row)
        
del csv_file_2[0] #enlever première ligne

L = []
for data in pet0 : 
    liste = []
    pat_id = data[1]
    for row in csv_file_2 : 
        if pat_id == row[1] : 
            liste.append(pat_id)
            liste.append(row[4])
            break
    L.append(liste)

m = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
c = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
for sub in L :
    date = sub[1].split('_')
    #print(date)
    month = date[2].split(' ')[0]
    day = date[2].split(' ')[1]
    if len(date) == 5 :
        year = date[3]
    else : year = date[-1].split('-')[0].split('.')[0]
    index = m.index(month)
    new = []
    new.append(year)
    new.append(c[index])
    new.append(day)
    #print(new)
    sub.append('_'.join(new))
    
for i in range(len(L)):
    pet0[i].append(L[i][-1])

In [ ]:
for nifti in pet0 : 
    try : 
        print(pet0.index(nifti))
        mask_path = nifti[-2]
        pet_path = nifti[-3]
        liste_center = get_center_of_mass(mask_path, thresh = True, pet_path = pet_path)
        d_max = calcul_distance_max(liste_center)
        nifti.append(d_max)
    except Exception as err : 
        print(nifti)
        print(err)

In [ ]:
nifti_directory = '/media/deeplearning/LACIE SHARE/AHL_NIFTI'
filename = 'AHL2011_dMax_v4.csv'

with open(os.path.join(nifti_directory, filename), 'w') as csv_file : 
    csv_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(["PATIENT ID", "DATE", "STUDY_UID", "D_MAX"])
    for row in pet0: 
        csv_writer.writerow([row[1], row[-2], row[2], row[-1]])
    for row in dataset : 
        csv_writer.writerow([row[0], row[1], row[2], row[3]])